### Data check and testing with minsearch

In [143]:
import pandas as pd

In [144]:
df = pd.read_csv('data/data.csv')

In [145]:
df

,book_name,author_name,subgenres,summary,rating,release_date
0,Dune,Frank Herbert,Space Opera; Political SF,"On the desert planet Arrakis, control over the...",Four and a half out of Five,1965
1,Neuromancer,William Gibson,Cyberpunk,A washed-out hacker is drawn into a dangerous ...,Four out of Five,1984
2,The Left Hand of Darkness,Ursula K. Le Guin,Social SF; Planetary SF,An envoy to the icy world of Winter must navig...,Four out of Five,1969
3,Foundation,Isaac Asimov,Social SF; Space Opera,"As a galactic empire declines, a Foundation is...",Four and a half out of Five,1951
4,The Hobbit,J.R.R. Tolkien,High Fantasy; Adventure,Bilbo Baggins sets out on a quest with dwarves...,Four and a half out of Five,1937
...,...,...,...,...,...,...
188,The Bonehunters,Steven Erikson,Epic Fantasy,Armies clash while hidden powers pull strings ...,Four and a half out of Five,2006
189,Reaper’s Gale,Steven Erikson,Epic Fantasy,The Malazan saga grows darker as betrayals and...,Four out of Five,2007
190,Toll the Hounds,Steven Erikson,Epic Fantasy,Death and fate converge in Darujhistan in a tr...,Four out of Five,2008
191,Dust of Dreams,Steven Erikson,Epic Fantasy,The armies march toward the final convergence ...,Four out of Five,2009


In [146]:
documents = df.to_dict(orient='records')

In [147]:
documents

[{'book_name': 'Dune',
  'author_name': 'Frank Herbert',
  'subgenres': 'Space Opera; Political SF',
  'summary': 'On the desert planet Arrakis, control over the spice melange triggers a clash of prophecy, politics, and power.',
  'rating': 'Four and a half out of Five',
  'release_date': 1965},
 {'book_name': 'Neuromancer',
  'author_name': 'William Gibson',
  'subgenres': 'Cyberpunk',
  'summary': 'A washed-out hacker is drawn into a dangerous AI heist across cyberspace and corporate intrigue.',
  'rating': 'Four out of Five',
  'release_date': 1984},
 {'book_name': 'The Left Hand of Darkness',
  'author_name': 'Ursula K. Le Guin',
  'subgenres': 'Social SF; Planetary SF',
  'summary': 'An envoy to the icy world of Winter must navigate a culture without fixed gender amid political tension.',
  'rating': 'Four out of Five',
  'release_date': 1969},
 {'book_name': 'Foundation',
  'author_name': 'Isaac Asimov',
  'subgenres': 'Social SF; Space Opera',
  'summary': "As a galactic empire 

In [148]:
import minsearch

# Create and fit the index
index = minsearch.Index(
    text_fields=['book_name', 'author_name', 'subgenres', 'summary', 'rating'],
    keyword_fields=[]
)
index.fit(documents)

In [149]:
query = "Which books are written by Robert Jordan?"
results = index.search(query, num_results=10)

In [150]:
results

[{'book_name': 'The Eye of the World',
  'author_name': 'Robert Jordan',
  'subgenres': 'Epic Fantasy',
  'summary': 'Rand al’Thor and his friends flee their village and are drawn into a world-spanning struggle with the Dark One.',
  'rating': 'Four out of Five',
  'release_date': 1990},
 {'book_name': 'The Great Hunt',
  'author_name': 'Robert Jordan',
  'subgenres': 'Epic Fantasy',
  'summary': 'Rand and his companions pursue the Horn of Valere, while destiny tightens its grip.',
  'rating': 'Four and a half out of Five',
  'release_date': 1990},
 {'book_name': 'The Dragon Reborn',
  'author_name': 'Robert Jordan',
  'subgenres': 'Epic Fantasy',
  'summary': 'Prophecy takes shape as Rand embraces his fate as the Dragon Reborn.',
  'rating': 'Four and a half out of Five',
  'release_date': 1991},
 {'book_name': 'The Path of Daggers',
  'author_name': 'Robert Jordan',
  'subgenres': 'Epic Fantasy',
  'summary': 'Armies clash as prophecy and destiny move the heroes toward Tarmon Gai’don

In [151]:
def search(query):
    boost = {}

    results = index.search(
        query=query,
        filter_dict={},
        boost_dict=boost,
        num_results=10
    )

    return results

### Configuring Ollama

In [156]:
prompt_template = """
You are an assistant for helping people decide on which fantasy and sci-fi books to read. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: {context}
""".strip()
    
entry_template = """
book_name: {book_name}
author_name: {author_name}
subgenres: {subgenres}
summary: {summary}
rating: {rating}
release_date: {release_date}
""".strip()

def build_prompt(query, search_results):
    context = ""
    
    for doc in search_results:
        context = context + entry_template.format(**doc) + "\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [166]:
def llm(prompt):
    from ollama import chat
    from ollama import ChatResponse

    response: ChatResponse = chat(
        
        model='llama3.2:latest', 
        messages=[ {'role': 'user','content': prompt}]
        )
    
    return response.message.content

In [175]:
query = 'What is the release order of Steven Erikson books?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [176]:
answer = rag(query)
print(answer)

To determine the release order of Steven Erikson's books, we can look at the release dates provided in the context:

1. Gardens of the Moon (1999)
2. Deadhouse Gates (2000)
3. Memories of Ice (2001)
4. House of Chains (2002)
5. Midnight Tides (2004)
6. Reaper’s Gale (2007)
7. Toll the Hounds (2008)
8. The Bonehunters (2006) * corrected order
9. Gardens of the Moon was already established as #1 and Deadhouse Gates is at #2; this now places Memories of Ice at #3, House of Chains at #4, followed by Midnight Tides, Reaper's Gale, Toll the Hounds.
10. The Crippled God (2011)
